# Advanced diagnostics: real vs synthetic electricity windows

This notebook runs STL decomposition, change-point detection, and lagged cross-correlation on example windows and saves figures + a JSON summary to `outputs/figures/diagnostics/`.


In [1]:
import os
os.chdir(r"C:\BDA_CEP_Part-2")
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt

# required libs: statsmodels, ruptures
try:
    from statsmodels.tsa.seasonal import STL
except Exception as e:
    raise SystemExit("Please install statsmodels (pip install statsmodels). Error: " + str(e))
try:
    import ruptures as rpt
except Exception as e:
    raise SystemExit("Please install ruptures (pip install ruptures). Error: " + str(e))

# reproducibility for any shuffles/sampling (not strictly needed here but handy)
SEED = 42
np.random.seed(SEED)


## Paths, output directory, and file checks


In [2]:
OUT_DIR = "outputs/figures/diagnostics"
os.makedirs(OUT_DIR, exist_ok=True)

DATA_DIR = "data/processed/electricity"
SYN_PATH = "outputs/synth/synth_electricity_2000w.npy"   # inverse-scaled synth

# quick sanity checks
for p in [
    os.path.join(DATA_DIR, "test.npy"),
    os.path.join(DATA_DIR, "scalers.pkl"),
    os.path.join(DATA_DIR, "features.txt"),
    SYN_PATH
]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Required file not found: {p}")
print("All required files found. Outputs will go to:", OUT_DIR)


All required files found. Outputs will go to: outputs/figures/diagnostics


## Load data and (un)scale


In [5]:
# load scaled real and scaler (we'll inverse transform real into original units)
real_scaled = np.load(os.path.join(DATA_DIR, "test.npy"))    # scaled
with open(os.path.join(DATA_DIR, "scalers.pkl"), "rb") as f:
    scalers = pickle.load(f)  # This is a list of scalers

# convert scaled real back to original units for human-meaningful plots
n_real, T, D = real_scaled.shape
real_orig = np.zeros_like(real_scaled)

# Apply inverse transform for each feature
for feature_idx in range(D):
    feature_data = real_scaled[:, :, feature_idx].reshape(-1, 1)
    orig_feature = scalers[feature_idx].inverse_transform(feature_data)
    real_orig[:, :, feature_idx] = orig_feature.reshape(n_real, T)

# load synth (inverse-scaled already in original units)
synth_inv = np.load(SYN_PATH)

# create scaled synth too (for any comparisons in scaled space)
synth_scaled = np.zeros_like(synth_inv)

# Apply transform for each feature to scale synth data
for feature_idx in range(D):
    feature_data = synth_inv[:, :, feature_idx].reshape(-1, 1)
    scaled_feature = scalers[feature_idx].transform(feature_data)
    synth_scaled[:, :, feature_idx] = scaled_feature.reshape(synth_inv.shape[0], synth_inv.shape[1])

FEATURE_NAMES = open(os.path.join(DATA_DIR, "features.txt")).read().splitlines()
print("Loaded shapes -> real_scaled:", real_scaled.shape, "real_orig:", real_orig.shape, "synth_inv:", synth_inv.shape)

Loaded shapes -> real_scaled: (216, 168, 7) real_orig: (216, 168, 7) synth_inv: (2000, 168, 7)


## Helper functions
- STL decomposition + plot
- change-point detection (ruptures PELT with RBF)
- plotting change-points
- lagged cross-correlation


In [2]:
PERIOD = 24   # hourly data with daily seasonality
CHANGEPOINT_PEN = 10  # penalty for ruptures; adjust if too many/few CPs
MAX_LAG = 48  # for lagged cross-correlation (hours)

def stl_and_plot(series, title_prefix, out_png, period=PERIOD):
    """Compute STL and save a figure. Returns small metadata dict."""
    stl = STL(series, period=period, robust=True)
    res = stl.fit()
    fig, axes = plt.subplots(4, 1, figsize=(10, 7), sharex=True)
    axes[0].plot(res.trend); axes[0].set_title(f"{title_prefix} — Trend")
    axes[1].plot(res.seasonal); axes[1].set_title(f"{title_prefix} — Seasonal")
    axes[2].plot(res.resid); axes[2].set_title(f"{title_prefix} — Residual")
    axes[3].plot(series); axes[3].set_title(f"{title_prefix} — Original")
    plt.tight_layout()
    fig.savefig(out_png, dpi=200, bbox_inches="tight")
    plt.close(fig)
    return {"trend_mean": float(np.nanmean(res.trend)), "seasonal_std": float(np.nanstd(res.seasonal))}

def detect_changepoints(series, pen=CHANGEPOINT_PEN):
    """Return list of change-point indices using PELT with RBF."""
    algo = rpt.Pelt(model="rbf").fit(series)
    try:
        cps = algo.predict(pen=pen)
    except Exception:
        # fallback to a small fixed number of breakpoints
        cps = algo.predict(n_bkps=3)
    return cps

def plot_changepoints(series, cps, title, out_png):
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(series, label=title)
    for cp in cps[:-1]:
        ax.axvline(cp, color='red', linestyle='--', alpha=0.7)
    ax.set_title(title + " (change-points dashed)")
    ax.legend()
    plt.tight_layout()
    fig.savefig(out_png, dpi=200, bbox_inches="tight")
    plt.close(fig)

def lagged_corr(x, y, max_lag=MAX_LAG):
    lags = list(range(-max_lag, max_lag + 1))
    corrs = []
    for lag in lags:
        if lag < 0:
            a = x[:lag]; b = y[-lag:]
        elif lag > 0:
            a = x[lag:]; b = y[:-lag]
        else:
            a = x; b = y
        if a.size == 0 or b.size == 0:
            corrs.append(0.0)
        else:
            c = np.corrcoef(a, b)[0, 1]
            corrs.append(0.0 if np.isnan(c) else float(c))
    return np.array(lags), np.array(corrs)


## Run diagnostics for example windows
- STL + CPD for all features in each example window
- Lagged cross-correlation for selected feature pairs
- Summary JSON with CP counts will be saved in OUT_DIR


In [7]:
EXAMPLES = 3  # number of windows to analyze side-by-side (first EXAMPLES windows)
feature_pairs = [(0, 1), (0, 2)]   # pairs to compute lagged cross-correlation
num_examples = min(EXAMPLES, n_real, synth_inv.shape[0])

summary = {"windows": []}

for i in range(num_examples):
    win_summary = {"index": int(i), "features": {}}
    for fi in range(D):
        fname = FEATURE_NAMES[fi] if fi < len(FEATURE_NAMES) else f"f{fi}"
        # pick series for real and synth (original units)
        s_real = real_orig[i, :, fi]
        s_synth = synth_inv[i, :, fi]  # synth_inv already in original units

        # 1) STL decomposition and plot
        out_r = os.path.join(OUT_DIR, f"stl_real_win{i}_feat{fi}_{fname}.png")
        out_s = os.path.join(OUT_DIR, f"stl_synth_win{i}_feat{fi}_{fname}.png")
        meta_r = stl_and_plot(s_real, f"Real win{i} {fname}", out_r)
        meta_s = stl_and_plot(s_synth, f"Synth win{i} {fname}", out_s)
        win_summary["features"][fname] = {"stl_real": meta_r, "stl_synth": meta_s}

        # 2) change-point detection
        cps_r = detect_changepoints(s_real)
        cps_s = detect_changepoints(s_synth)
        plot_changepoints(s_real, cps_r, f"Real win{i} {fname}", os.path.join(OUT_DIR, f"cp_real_win{i}_feat{fi}_{fname}.png"))
        plot_changepoints(s_synth, cps_s, f"Synth win{i} {fname}", os.path.join(OUT_DIR, f"cp_synth_win{i}_feat{fi}_{fname}.png"))
        win_summary["features"][fname].update({"cps_real": cps_r, "cps_synth": cps_s})

    # 3) lagged cross-correlations for chosen feature pairs
    win_summary["lagged_corr"] = {}
    for (a, b) in feature_pairs:
        x_r = real_orig[i, :, a]; y_r = real_orig[i, :, b]
        x_s = synth_inv[i, :, a]; y_s = synth_inv[i, :, b]
        lags, corr_r = lagged_corr(x_r, y_r)
        _, corr_s = lagged_corr(x_s, y_s)
        # plot
        fig, ax = plt.subplots(figsize=(8, 3))
        ax.plot(lags, corr_r, label=f"Real {FEATURE_NAMES[a]} vs {FEATURE_NAMES[b]}")
        ax.plot(lags, corr_s, linestyle='--', label=f"Synth {FEATURE_NAMES[a]} vs {FEATURE_NAMES[b]}")
        ax.axvline(0, color='k', linewidth=0.5)
        ax.set_xlabel("lag")
        ax.set_ylabel("correlation")
        ax.legend()
        fig_path = os.path.join(OUT_DIR, f"lagcorr_win{i}_feat{a}_{a}_feat{b}_{b}.png")
        fig.savefig(fig_path, dpi=200, bbox_inches="tight")
        plt.close(fig)
        win_summary["lagged_corr"][f"{FEATURE_NAMES[a]}__{FEATURE_NAMES[b]}"] = {
            "lags": lags.tolist(),
            "corr_real": corr_r.tolist(),
            "corr_synth": corr_s.tolist(),
            "plot": fig_path
        }

    summary["windows"].append(win_summary)

# Save summary JSON
summary_path = os.path.join(OUT_DIR, "diagnostics_summary.json")
with open(summary_path, "w") as ff:
    json.dump(summary, ff, indent=2)

print("Diagnostics complete. Plots & JSON summary saved to:", OUT_DIR)


Diagnostics complete. Plots & JSON summary saved to: outputs/figures/diagnostics


## Notes & tuning tips
- `PERIOD` (24) assumes hourly data with daily seasonality; change if your sampling is different.
- `CHANGEPOINT_PEN` is the ruptures penalty: increase to get fewer cps, decrease to get more.
- If images are large or you want interactive exploration, consider plotting inline and inspecting specific windows before saving.
- For a quicker debug run, reduce `EXAMPLES` or `D` (feature loop).
